## **Copy by Value vs Copy by Reference**


### **Copy by Value**

Quando si passa una **variabile di tipo primitivo** (int, float, double, bool...) come parametro di una funzione, C# **crea una copia** del suo valore.
Ciò significa che **modificare la variabile locale nella funzione non cambia l’originale**.

**Esempio:**

```csharp
void Incrementa(int x)
{
    x++;
    Debug.Log("Il valore di x = " + x);
}

void Start()
{
    int a = 5;
    Incrementa(a);
    Debug.Log("Il valore di a = " + a); // Stampa 5
}
```

➡️ La variabile `a` non viene modificata, perché `x` è solo una copia del suo valore.
Questo comportamento si chiama **Copy by Value**.

---

### **Copy by Reference (ref)**

Per **non copiare** il valore ma lavorare **sulla variabile originale**, si usa la parola chiave `ref`.
Essa permette alla funzione di **modificare direttamente la variabile passata**.

**Esempio:**

```csharp
void IncrementaConRef(ref int x)
{
    x++;
}

void Start()
{
    int a = 5;
    IncrementaConRef(ref a);
    Debug.Log("Il valore di a = " + a); // Stampa 6
}
```

➡️ `ref` significa **reference**, cioè riferimento: il parametro non è più una copia, ma un “alias” della variabile originale.

---

### **Copy by Reference (out)**

La parola chiave `ref` si può usare **solo se la variabile è già inizializzata**.
Se invece la funzione deve **inizializzare** una variabile (cioè assegnarle il primo valore), si usa la parola chiave **`out`**.

**Esempio:**

```csharp
void InizializzaVariabile(out int x)
{
    x = 5; // Deve essere assegnato un valore
}

void Start()
{
    int n; // Dichiarata ma non inizializzata
    InizializzaVariabile(out n);
    Debug.Log("Il valore di n = " + n); // Stampa 5
}
```

➡️ Con `out`, la variabile viene **obbligatoriamente valorizzata dentro la funzione** prima che termini.

---

### **In sintesi**

| Tipo di passaggio                        | Parola chiave | Requisito                   | Effetto sulla variabile originale    |
| ---------------------------------------- | ------------- | --------------------------- | ------------------------------------ |
| **Copy by Value**                        | (nessuna)     | Variabile definita          | **Non modificata**                   |
| **Copy by Reference**                    | `ref`         | Variabile già inizializzata | **Modificata**                       |
| **Copy by Reference (inizializzazione)** | `out`         | Variabile non inizializzata | **Inizializzata dentro la funzione** |

## Regole pratiche

* **Vuoi aggiornare** una variabile esistente? → `ref`.
* **Vuoi creare il primo valore** all’interno della funzione? → `out`.
* Non ti serve toccare l’originale? → nessuna parola chiave (copia).
* Con **classi** (reference type): per **modificare i campi** basta il passaggio normale; usa `ref` solo se vuoi **sostituire l’oggetto** stesso.

---



## Idee chiave (brevi)

* **Bit/Byte**: 8 bit = 1 byte. Con i byte rappresenti numeri, caratteri, ecc.
* **RAM**: memoria volatile dove gira il programma. Veloce, si svuota allo spegnimento.
* **Memoria di massa (SSD/HDD)**: persistente, lenta; i programmi e i dati “vivono” qui quando non sono in esecuzione.

### Stack (pila)

* Zona della RAM usata per **chiamate di funzione**.
* Contiene **parametri**, **variabili locali** (soprattutto value type) e **indirizzo di ritorno**.
* Cresce e si riduce in modo **LIFO** a ogni call/return → **molto veloce**.
* **Limitato**: troppa ricorsione o troppi dati locali → **StackOverflow**.
* Per i **reference type** (classi), sullo stack sta **solo il riferimento**; l’oggetto sta sull’heap.

### Heap

* Zona della RAM per **oggetti dinamici**: classi, **array**, **string**, value type “boxed”, lambda catturate, ecc.
* Allocazione con `new`.
* Gestito dal **Garbage Collector** (GC) → libera automaticamente gli oggetti non più referenziati.
* **Più flessibile** ma con **costo** (allocazione e pause GC).

### Riepilogo pratico

* **Value type (struct, int, float, bool, ecc.)**: spesso su **stack** quando sono locali; **inline** dentro oggetti/array; finiscono su heap se “boxed”.
* **Reference type (class, array, string)**: l’**istanza** sta sempre su **heap**; la variabile contiene un **riferimento** (sullo stack o dentro un altro oggetto).

---


## Riassunto visivo

Ogni funzione chiamata → “spinge” un nuovo blocco nello stack.
Quando finisce → il blocco viene tolto.

| Fase            | Stack (dall’alto verso il basso) |
| --------------- | -------------------------------- |
| Start() inizia  | Start                            |
| Chiama Func()   | Func → Start                     |
| Chiama Somma()  | Somma → Func → Start             |
| Somma() termina | Func → Start                     |
| Func() termina  | Start                            |
| Fine            | — vuoto —                        |

---




## Idee chiave sulle slide (Heap)

* Anche l’**Heap** fa parte della **RAM**, ma è l’area dove vivono **oggetti, array, stringhe** — cioè tutto ciò che richiede **allocazione dinamica**.
* Le variabili che usi nel codice per questi oggetti contengono **solo un riferimento (indirizzo in memoria)**, non l’oggetto stesso.
  → Puoi immaginarle come **frecce** che puntano all’oggetto sull’heap.
* Quando passi un array o un oggetto a una funzione **senza `ref`**, stai passando **una copia del riferimento**.
  Quindi entrambe le variabili (nel chiamante e nella funzione) **puntano allo stesso oggetto**, e modificare i suoi contenuti dentro la funzione **si riflette anche fuori**.
* Quando la funzione che teneva una freccia termina, quella **freccia sparisce**.
  Se **tutte le frecce** che puntavano a quell’oggetto scompaiono, l’oggetto rimane **isolato** (non più accessibile).
* Un oggetto isolato viene eliminato automaticamente dal **Garbage Collector (GC)**, un sistema che **ripulisce la memoria** liberando gli oggetti non più usati.
* In altri linguaggi (come C o C++) questo processo non è automatico, e spetta al programmatore liberare manualmente la memoria.
  In C# e Unity, invece, ci pensa il GC, anche se in certi contesti ad alte prestazioni (come i giochi) può comunque avere un impatto.
* L’intervento del GC **può causare brevi pause** nel programma, perché deve analizzare la memoria e liberarla.
  Più dati ha da controllare, maggiore sarà il suo impatto sulle performance.
* Per evitare rallentamenti, è buona pratica **ridurre le allocazioni inutili** e **riutilizzare oggetti** quando possibile.
  → Ad esempio, in Unity conviene **riciclare** istanze di oggetti (come proiettili o particelle) invece di distruggerle e ricrearle ogni volta — il cosiddetto **object pooling**.

---




## **Enumeratori (enum)** — Idee chiave

Un **enumeratore** è un tipo di dato personalizzato che ti permette di elencare una **serie di valori con nomi precisi**, invece di usare numeri o stringhe.

### Perché usarli

* Se usi **int**, devi ricordarti che ad esempio `0 = FUOCO`, `1 = TERRA`, ecc. (facile sbagliarsi).
* Se usi **string**, rischi errori di battitura (`"Foco"` invece di `"Fuoco"`).
* Con un **enum**, invece, puoi definire una lista di valori **stabili e leggibili**, e il compilatore ti protegge dagli errori.

---

### Definizione base

Esempio di enumeratore che definisce gli elementi di un gioco:

```csharp
public enum ELEMENTI
{
    FUOCO,
    TERRA,
    ACQUA,
    ARIA,
    GHIACCIO,
    FULMINE
}
```

> Non è obbligatorio scriverli su più righe; puoi anche farlo in una sola:
> `public enum ELEMENTI { FUOCO, TERRA, ACQUA, ARIA, GHIACCIO, FULMINE }`

---

### Uso di un enum

Dopo averlo definito, puoi usarlo per creare variabili di quel tipo:

```csharp
ELEMENTI mioElemento = ELEMENTI.FUOCO;
```

Spiegazione:

* `ELEMENTI` → è il **tipo di dato**.
* `mioElemento` → è la **variabile**.
* `ELEMENTI.FUOCO` → è il **valore** scelto dall’enum.

Il compilatore riconosce solo i nomi definiti dentro `ELEMENTI`.
Se scrivi per errore `ELEMENTI.FOCO`, ti segnala **errore**.

---

### Scope degli enum (dove dichiararli)

Puoi dichiarare un `enum`:

* **fuori da qualsiasi classe** → accessibile da tutto il codice.
* **dentro una classe** → visibile solo all’interno di quella classe.

Esempio:

```csharp
public class Enemy
{
    public enum STATE { IDLE, CHASE, ATTACK, DEFEATED }

    public int lifePoints;
    public STATE state = STATE.IDLE;
}
```

Qui `STATE` è un tipo valido **solo dentro la classe `Enemy`**.
Quindi potrai scrivere:

```csharp
Enemy e = new Enemy();
e.state = Enemy.STATE.ATTACK;
```

---

### Perché sono utili

* Riducono gli **errori di digitazione**.
* Rendono il codice **più leggibile**.
* Si usano spesso per gestire **stati di gioco**, **tipi di nemici**, **elementi**, **azioni**, ecc.

---

### In sintesi

| Tipo di variabile | Svantaggi                         | Con enum                                 |
| ----------------- | --------------------------------- | ---------------------------------------- |
| `int`             | Devi ricordare i numeri associati | Usa nomi chiari e leggibili              |
| `string`          | Possibili errori di scrittura     | Nomi fissi e controllati dal compilatore |
| `enum`            | —                                 | Sicuro, leggibile, facile da mantenere   |

---

# Dove posso dichiarare un enum?

## 1) Dentro una **classe** (enum “annidato”)

* L’enum **appartiene** alla classe.
* Da fuori devi qualificarlo con `NomeClasse.NomeEnum`.

```csharp
public class Enemy
{
    public enum STATE { IDLE, CHASE, ATTACK, DEFEATED }

    public int lifePoints;
    public STATE state = STATE.IDLE;
}

// Uso da un’altra classe:
public class GameMode
{
    public Enemy.STATE enemyState = Enemy.STATE.CHASE;
}
```

**Quando conviene:** lo stato è concettualmente legato a quella classe (riduci collisioni di nomi).

---

## 2) Dentro un **namespace**

* L’enum è visibile ovunque **nel namespace**; fuori va qualificato col nome del namespace.
* Puoi abbreviare con `using`.

```csharp
namespace GameTypes
{
    public enum ELEMENTS { FIRE, LIGHTNING, ICE, EARTH, WIND, FORCE }
}

// Uso qualificato
public class MagicA
{
    public GameTypes.ELEMENTS element = GameTypes.ELEMENTS.FIRE;
}

// Con using, più comodo
using GameTypes;
public class MagicB
{
    public ELEMENTS element = ELEMENTS.FIRE;
}
```

**Quando conviene:** enum “condivisi” da più classi/moduli.

---

## 3) **Globale** (fuori da classi e namespace)

* Accessibile ovunque **senza prefissi**.
* Rischi più alti di **collisione dei nomi** con librerie esterne.

```csharp
public enum STATE { IDLE, CHASE, ATTACK, DEFEATED }

public class Enemy  { public STATE state = STATE.IDLE; }
public class GameMode { public STATE enemyState = STATE.CHASE; }
```

**Quando conviene:** in progetti piccoli o per valori davvero universali e ben nominati.

---

# Note pratiche e buone norme

* **Visibilità:** l’enum deve essere `public` se vuoi usarlo fuori dal punto in cui è dichiarato.
* **Naming:** usa nomi **univoci e specifici** (`EnemyState`, `SpellElement`) per evitare conflitti.
* **Underlying type:** per default è `int`, ma puoi specificarlo:

  ```csharp
  public enum DamageType : byte { Physical, Fire, Ice }
  ```
* **Valori espliciti:**

  ```csharp
  public enum Rarity { Common = 1, Rare = 3, Epic = 5 }
  ```
* **Unity Inspector:** un campo `public` (o `[SerializeField]`) di tipo enum appare come **dropdown**.
  Anche gli enum annidati funzionano se sia l’enum sia il campo sono `public`.

---



## Cosa succede dietro le quinte

* Un `enum` è, internamente, un **numero intero** (per default `int`).
* Se **non** specifichi i numeri, C# li assegna in automatico partendo da **0** e incrementando di **1**.

```csharp
public enum ELEMENTS
{
    FIRE,       // 0
    LIGHTNING,  // 1
    ICE,        // 2
    EARTH,      // 3
    WIND,       // 4
    FORCE       // 5
}
```

## Il problema dello “slittamento”

Se **inserisci** un nuovo valore in mezzo (es. `WATER` tra `ICE` ed `EARTH`) e non hai numeri espliciti, **tutti i successivi si spostano**:

* `EARTH` da 3 → 4, `WIND` da 4 → 5, ecc.
* Questo può rompere **scene/serializzazioni** in Unity o **salvataggi**.

## Come evitarlo

Assegna **numeri espliciti**. Così puoi aggiungere voci nuove senza spostare i vecchi valori:

```csharp
public enum ELEMENTS
{
    FIRE      = 0,
    LIGHTNING = 1,
    ICE       = 2,
    EARTH     = 3,
    WIND      = 4,
    FORCE     = 5,

    // nuovo in futuro:
    WATER     = 6   // non altera i precedenti
}
```

Puoi anche “saltare” numeri se ti serve:

```csharp
public enum ELEMENTS
{
    FIRE = 0,
    LIGHTNING = 1,
    ICE = 2,
    WATER = 6,   // salto intenzionale
    EARTH = 3,
    WIND = 4,
    FORCE = 5
}
```

## Cast e conversioni utili

Essendo numeri, puoi **castare**:

```csharp
ELEMENTS e = ELEMENTS.WIND;
int n = (int)e;                     // -> 4

int k = 1;
ELEMENTS fromInt = (ELEMENTS)k;     // -> LIGHTNING
```

## Extra pratici

* Puoi cambiare il **tipo sottostante** per risparmiare memoria:

  ```csharp
  public enum DamageType : byte { Physical = 0, Fire = 1, Ice = 2 }
  ```
* In Unity: se l’enum è serializzato, preferisci **valori espliciti** per stabilità tra versioni.

---


Riassunto rapido (Unity/C# — enum in Inspector)

* Un `enum` pubblico (o `private` con `[SerializeField]`) appare come menu a tendina nell’Inspector: comodo per impostare stati/varianti senza “magic numbers”.
* Dai sempre un valore esplicito e un `None = 0`, non riordinare in futuro (rompe i prefab/salvataggi).
* Usa `switch` o tabelle per convertire l’enum in logica/dati (danni, colori, VFX, ecc.).
* Se devi selezionare più opzioni insieme, valuta i **flag** (`[System.Flags]`).

Esempio pulito

```csharp
public enum Element
{
    None = 0,
    Fire = 1,
    Lightning = 2,
    Ice = 3,
    Earth = 4,
    Wind = 5,
    Force = 6
}

public class Magic : MonoBehaviour
{
    [SerializeField] private int damage = 100;
    [SerializeField] private Element element = Element.Lightning;

    public int GetDamageAgainst(Element target)
    {
        // Esempio semplice: moltiplicatori per accoppiate
        float multiplier = 1f;
        switch (element)
        {
            case Element.Fire:
                multiplier = (target == Element.Ice) ? 1.5f : 1f;
                break;
            case Element.Lightning:
                multiplier = (target == Element.Wind) ? 1.5f : 1f;
                break;
            // …altri casi
            default:
                multiplier = 1f;
                break;
        }
        return Mathf.RoundToInt(damage * multiplier);
    }
}
```

Variant con più scelte (mask)

```csharp
[System.Flags]
public enum ElementMask
{
    None     = 0,
    Fire     = 1 << 0,
    Lightning= 1 << 1,
    Ice      = 1 << 2,
    Earth    = 1 << 3,
    Wind     = 1 << 4,
    Force    = 1 << 5
}

public class SpellResistances : MonoBehaviour
{
    // In molte versioni di Unity un enum con [Flags] appare come checklist
    [SerializeField] private ElementMask resistances = ElementMask.None;

    public bool IsResistantTo(Element e) => (resistances & (ElementMask)(1 << ((int)e - 1))) != 0;
}
```

Best practice veloci

* Evita di rinominare/riordinare i membri dopo averli usati in scene/prefab.
* Quando serializzi su file, salva il **nome** dell’enum (stringa) se prevedi cambi futuri.
* Se serve mostrare l’enum ma tenerlo incapsulato, usa `[SerializeField] private` + proprietà `public` in sola lettura.

---


## Switch e Enum

### Descrizione

Lo **switch** è una struttura di controllo del flusso simile a una serie di `if / else if`, ma più ordinata e leggibile quando si devono gestire **più casi** di uno stesso valore.
È particolarmente utile quando una variabile può assumere un set limitato di valori.

### Sintassi base

```csharp
int mese = 2;

switch (mese)
{
    case 1:
        Debug.Log("Gennaio");
        break;
    case 2:
        Debug.Log("Febbraio");
        break;
    case 10:
        Debug.Log("Ottobre");
        break;
    default:
        Debug.Log("Mese non valido");
        break;
}
```

**Note importanti:**

* Ogni `case` termina con **due punti (`:`)**, non con `;`.
* Il comando `break` è necessario per uscire dallo `switch` dopo l’esecuzione di un caso.
* Il blocco `default` viene eseguito se nessun caso corrisponde.

---

### Switch con Enum

Gli **enum** rendono il codice più leggibile e meno soggetto a errori, soprattutto quando i valori rappresentano stati o categorie.

```csharp
public enum STATE
{
    IDLE,
    CHASE,
    DEFEATED
}

public class Enemy : MonoBehaviour
{
    public STATE state = STATE.CHASE;

    void Update()
    {
        switch (state)
        {
            case STATE.IDLE:
                Debug.Log("Resto fermo");
                break;
            case STATE.CHASE:
                Debug.Log("Inseguo il player");
                break;
            case STATE.DEFEATED:
                Debug.Log("Animazione di sconfitta");
                break;
        }
    }
}
```

### Vantaggi

* Codice più **pulito e leggibile** rispetto a molti `if / else`.
* Gestione ordinata di **stati e comportamenti** (ottimo per piccole **state machine**).
* Perfetto da combinare con **enum pubblici**, visibili anche nell’**Inspector** di Unity.

---




## Namespace

### Descrizione

La parola chiave `namespace` definisce un **ambito logico** che raggruppa elementi di codice (classi, strutture, funzioni, ecc.) correlati tra loro.
Serve per **organizzare** il codice e per **evitare conflitti di nomi** tra componenti che hanno la stessa denominazione ma appartengono a contesti diversi.

### Sintassi base

```csharp
namespace NomeDelNamespace
{
    class ComponenteNelNamespace : MonoBehaviour
    {
        void Start()
        {
            Debug.Log("Componente appartenente a un namespace personalizzato");
        }
    }
}
```

---

### A cosa serve

* Permette di **riutilizzare nomi** uguali per classi diverse senza creare errori.
* Il compilatore riconosce ogni classe dal suo percorso completo (es. `UnityEngine.Vector3` o `System.Numerics.Vector3`).
* È fondamentale in progetti **modulari o condivisi**, dove possono coesistere librerie di terze parti.

---

### Quando è utile

* Quando importiamo librerie esterne (es. **Unity**, **.NET**, **plugin personalizzati**).
* Quando sviluppiamo codice che potrebbe essere **riutilizzato in più progetti**.
* Quando vogliamo **organizzare logicamente** il progetto in sottosezioni (es. `Game.Core`, `Game.Audio`, `Game.AI`).

---

### Esempio pratico

```csharp
// File 1
namespace Gioco.Audio
{
    public class SoundManager : MonoBehaviour
    {
        public void PlaySound() => Debug.Log("Suono riprodotto!");
    }
}

// File 2
namespace Gioco.AI
{
    public class Enemy : MonoBehaviour
    {
        void Start()
        {
            Debug.Log("Nemico generato nel namespace AI");
        }
    }
}
```

---

### Esempio di conflitto evitato

Unity e C# possiedono entrambi un tipo chiamato `Vector3`:

* `UnityEngine.Vector3` (usato per le coordinate 3D nei giochi)
* `System.Numerics.Vector3` (usato per calcoli matematici generici)

Grazie ai **namespace**, entrambi possono coesistere:

```csharp
UnityEngine.Vector3 position = new UnityEngine.Vector3(0, 1, 0);
System.Numerics.Vector3 mathVector = new System.Numerics.Vector3(1, 2, 3);
```

---

### Best practice

* Usa sempre **nomi chiari e gerarchici**, es. `Game.UI.Menu` invece di `Menu`.
* Evita di inserire troppe classi nello stesso namespace.
* Quando importi librerie esterne, ricorda che il `using` in cima al file serve per richiamare il namespace:

  ```csharp
  using UnityEngine;
  using System.Collections;
  ```

---



## Come fa il codice a capire quale usare?

Quando più namespace contengono tipi con lo stesso nome (es. `Vector3`), il compilatore deve sapere **a quale versione ci riferiamo**.
Ci sono **due modi principali** per indicarlo:

---

### 1. Specificare il percorso completo del tipo

Possiamo dichiarare una variabile scrivendo il **percorso completo** (namespace + nome del tipo):

```csharp
UnityEngine.Vector3 vettoreA;   // Usa il Vector3 di Unity
System.Numerics.Vector3 vettoreB; // Usa il Vector3 di System.Numerics
```

Questo metodo è esplicito e chiaro, ma può rendere il codice più verboso.

---

### 2. Usare la direttiva `using`

Scrivendo in cima al file:

```csharp
using UnityEngine;
```

Diciamo al compilatore di cercare prima i tipi dentro **quel namespace**.
Possiamo includere più namespace, uno per riga:

```csharp
using UnityEngine;
using System.Numerics;
```

---

### Attenzione!

Se rimuoviamo la riga `using UnityEngine;`, Unity **non riconoscerà più `MonoBehaviour`**, perché quella classe appartiene al namespace `UnityEngine`.

Perciò, o scriviamo:

```csharp
using UnityEngine;

public class NomeMiaComponente : MonoBehaviour
{
    ...
}
```

Oppure specifichiamo esplicitamente:

```csharp
public class NomeMiaComponente : UnityEngine.MonoBehaviour
{
    ...
}
```

---

### Conflitti tra namespace

Se includiamo **entrambi**:

```csharp
using UnityEngine;
using System.Numerics;
```

e dichiariamo:

```csharp
Vector3 vettore;
```

Unity utilizzerà **quello del primo namespace riconosciuto** (in questo caso `UnityEngine`).
Se vogliamo invece usare quello di `System.Numerics`, dobbiamo scrivere il percorso completo:

```csharp
System.Numerics.Vector3 vettore;
```

---

### Esempio completo

```csharp
using UnityEngine;
using System.Numerics;

public class NamespaceExample : MonoBehaviour
{
    void Start()
    {
        UnityEngine.Vector3 posizione = new UnityEngine.Vector3(0, 1, 0);
        System.Numerics.Vector3 calcolo = new System.Numerics.Vector3(2, 3, 4);

        Debug.Log($"Posizione Unity: {posizione}");
        Debug.Log($"Vettore matematico: {calcolo}");
    }
}
```

---

### In sintesi

| Caso                              | Soluzione                                                                     |
| --------------------------------- | ----------------------------------------------------------------------------- |
| Due tipi con lo stesso nome       | Specifica il percorso completo                                                |
| Tipo non riconosciuto             | Aggiungi `using` corretto in cima                                             |
| Molti namespace in uso            | Evita conflitti mantenendo i `using` chiari                                   |
| Dubbi su quale usa il compilatore | Controlla il tooltip in Unity o Visual Studio (mostra il namespace effettivo) |

---




# **Modulo 2 – Lezione 6 — Sintesi completa**

## **Copy by Value e Copy by Reference**

### Copy by Value

Quando una variabile **di tipo primitivo** (int, float, bool...) viene passata a una funzione, C# **copia il suo valore**.
Modificare la variabile interna **non influisce sull’originale**.

```csharp
void Incrementa(int x) { x++; }
int a = 5;
Incrementa(a); // a resta 5
```

➡️ Si crea una **copia** indipendente del valore.

---

### Copy by Reference (`ref`)

Usando la parola chiave `ref`, si passa alla funzione **un riferimento** all’originale.

```csharp
void IncrementaConRef(ref int x) { x++; }
int a = 5;
IncrementaConRef(ref a); // a diventa 6
```

➡️ `ref` modifica **direttamente la variabile originale**.

---

### Copy by Reference (`out`)

Si usa `out` quando la variabile **non è ancora inizializzata** e deve ricevere il primo valore **dentro la funzione**.

```csharp
void Inizializza(out int n) { n = 10; }
int x;
Inizializza(out x); // x = 10
```

| Tipo di passaggio                    | Parola chiave | Requisito                   | Effetto                   |
| ------------------------------------ | ------------- | --------------------------- | ------------------------- |
| Copy by Value                        | —             | Variabile inizializzata     | Non modificata            |
| Copy by Reference                    | `ref`         | Variabile già inizializzata | Modificata                |
| Copy by Reference (inizializzazione) | `out`         | Variabile non inizializzata | Inizializzata in funzione |

---

## **Stack e Heap**

### Stack

* Area veloce della **RAM** dove vivono **variabili locali e parametri**.
* Funziona come una pila **LIFO**.
* Si riempie e svuota automaticamente a ogni chiamata di funzione.
* Se si esagera (ricorsione profonda o troppi dati), → **StackOverflow**.

### Heap

* Area della RAM dedicata agli **oggetti dinamici** (classi, array, stringhe...).
* La memoria viene gestita dal **Garbage Collector (GC)**.
* Più flessibile, ma più lenta rispetto allo stack.
* Se un oggetto **non ha più riferimenti**, viene eliminato automaticamente.

### In breve

* **Value type** → di solito su **stack**.
* **Reference type** → l’oggetto sta su **heap**, la variabile contiene solo un **riferimento**.
* Se spariscono tutti i riferimenti, il GC libera la memoria.

---

## **Enum (Enumeratori)**

### Cos’è

Un `enum` è un tipo di dato che rappresenta una **lista di valori con nome**, più chiara e sicura di int o stringhe.

```csharp
public enum ELEMENTI { FUOCO, TERRA, ACQUA, ARIA, GHIACCIO, FULMINE }
```

➡️ Evita errori e rende il codice leggibile.

### Usi

```csharp
ELEMENTI mioElemento = ELEMENTI.FUOCO;
```

* `ELEMENTI` → tipo di dato
* `mioElemento` → variabile
* `ELEMENTI.FUOCO` → valore

### Dove dichiararli

1. **Dentro una classe** → visibili solo all’interno.
2. **Dentro un namespace** → condivisi da più script.
3. **Globali** → accessibili ovunque (ma rischio di conflitti).

### Buone pratiche

* Assegna **valori espliciti** per evitare problemi di serializzazione in Unity.
* Usa nomi chiari (`EnemyState`, `SpellElement`).
* Un campo `public` di tipo enum appare come **dropdown** nell’Inspector.

### Esempio pratico

```csharp
public enum Element { None = 0, Fire, Lightning, Ice }

public class Magic : MonoBehaviour
{
    public Element element = Element.Fire;
}
```

---

## **Switch e Enum**

### Cos’è

`switch` è una struttura che semplifica una serie di `if / else if`, utile per gestire **più casi** di uno stesso valore.

```csharp
int mese = 2;
switch (mese)
{
    case 1: Debug.Log("Gennaio"); break;
    case 2: Debug.Log("Febbraio"); break;
    default: Debug.Log("Altro mese"); break;
}
```

### Con gli Enum

Perfetto per gestire **stati di gioco**:

```csharp
public enum STATE { IDLE, CHASE, DEFEATED }

switch (state)
{
    case STATE.IDLE: Debug.Log("Fermo"); break;
    case STATE.CHASE: Debug.Log("Inseguo"); break;
    case STATE.DEFEATED: Debug.Log("Sconfitto"); break;
}
```

➡️ Usato spesso per **state machine semplici** (IA, gameplay, animazioni).

---

## **Namespace**

### Cos’è

`namespace` serve a **raggruppare classi e funzioni correlate** in uno spazio dei nomi, evitando conflitti tra componenti con lo stesso nome.

```csharp
namespace Game.Audio
{
    public class SoundManager { }
}
```

### Perché serve

Permette di avere più classi con lo stesso nome in contesti diversi, come:

* `UnityEngine.Vector3`
* `System.Numerics.Vector3`

### Quando è utile

* In progetti grandi o modulari.
* Quando si usano **librerie esterne** o plugin.
* Per mantenere ordine e coerenza.

### Come specificare quale usare

1. **Percorso completo**

   ```csharp
   UnityEngine.Vector3 posizione;
   System.Numerics.Vector3 calcolo;
   ```

2. **Direttiva `using`**

   ```csharp
   using UnityEngine;
   Vector3 pos; // Usa quello di Unity
   ```

### Attenzione

Se togli `using UnityEngine;`, il compilatore non riconosce più `MonoBehaviour`.

➡️ Alternativa:

```csharp
public class Example : UnityEngine.MonoBehaviour { }
```

### In caso di conflitto

Se includi entrambi:

```csharp
using UnityEngine;
using System.Numerics;
```

C# userà il primo trovato.
Per l’altro, scrivi il percorso completo.

### Esempio

```csharp
using UnityEngine;
using System.Numerics;

public class NamespaceExample : MonoBehaviour
{
    void Start()
    {
        UnityEngine.Vector3 pos = new UnityEngine.Vector3(0, 1, 0);
        System.Numerics.Vector3 calc = new System.Numerics.Vector3(2, 3, 4);
    }
}
```

---

## **Schema riassuntivo finale**

| Concetto              | Parola chiave / Struttura       | Scopo principale                                |
| --------------------- | ------------------------------- | ----------------------------------------------- |
| **Copy by Value**     | —                               | Passa una copia del valore                      |
| **Copy by Reference** | `ref`, `out`                    | Modifica o inizializza l’originale              |
| **Stack / Heap**      | —                               | Gestione della memoria (temporanea vs dinamica) |
| **Enum**              | `enum`                          | Liste di valori leggibili e sicure              |
| **Switch**            | `switch (valore)`               | Controllo di flusso per casi multipli           |
| **Namespace**         | `namespace Nome` / `using Nome` | Organizzazione del codice ed evitare conflitti  |

---


